In [1]:
#!/usr/bin/env python
# coding: utf-8
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
# Purpose of script: To download wind turbine data from BEIS' Renewable Energy Planning Database and Wikipedia and merge them

# Import libraries from which to call functions e.g. urllib 
import urllib.request
import traceback
import pandas as pd  # library for data analysis
import requests  # library to handle requests
from bs4 import BeautifulSoup  # library to parse HTML documents
import pyproj
import traceback
from lat_lon_parser import parse
import geopy.distance

crs_british = pyproj.Proj(init='EPSG:27700')
crs_wgs84 = pyproj.Proj(init='EPSG:4326')

words_to_ignore = ['offshore', 'wind', 'farm', 'plant']

In [3]:
def find_commonwords_num(string1, string2, filters):
    """
    :param string1:
    :param string2:
    :param filters:
    :return:
    """
    try:
        filters = [x.lower() for x in filters]
        list1 = string1.lower().split()
        list2 = string2.lower().split()
        set2 = set(list2)
        f = lambda x: x in set2
        return len([x for x in list(filter(f, list1)) if x.lower() not in filters])
    except Exception:
        print(traceback.format_exc())


def convert_uk_grid_to_latlon(row):
    """
    :param x:
    :param y:
    :return:
    """
    try:
        x = row['X-coordinate']
        y = row['Y-coordinate']
        long, lat = pyproj.transform(crs_british, crs_wgs84, x, y)
        return (lat, long)
    except Exception as e:
        return (0, 0)


def formatted_string_to_latlon(geo_line):
    """
    :param geo_line:
    :return:
    """
    try:
        lat, lon = geo_line.split('/')[1].strip().split()
        lat = parse(lat)
        lon = parse(lon)
        return (lat, lon)
    except Exception as e:
        return (0, 0)


def get_distance(latlon1, latlon2):
    """
    :param latlon1:
    :param latlon2:
    :return:
    """
    try:
        return round(geopy.distance.geodesic(latlon1, latlon2).km, 2)
    except Exception as e:
        print("Exception in get_distance: {}".format(str(e)))
        print(traceback.format_exc())


In [4]:
def get_REPD_Wind_df():
    try:
        # Download Renewable Energy Planning Data (REPD) to location of script by default. Location of save can be specified elsewhere
        url = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1096108/repd-july-2022-corrected.csv'
        urllib.request.urlretrieve(url, './Renewable_Energy_Planning_Data.csv')

        # List of fields to retain from the REPD (REPD), with those not listed to be dropped
        fields = ['Ref ID', 'Record Last Updated (dd/mm/yyyy)',
                  'Operator (or Applicant)',
                  'Site Name',
                  'Technology Type',
                  'Installed Capacity (MWelec)',
                  'Turbine Capacity (MW)',
                  'No. of Turbines',
                  'Height of Turbines (m)',
                  'Development Status (short)',
                  'Address',
                  'County',
                  'Region',
                  'Country',
                  'Post Code',
                  'X-coordinate',
                  'Y-coordinate',
                  'Operational']

        # Read in the REPD csv file, specifying the encoding, while keeping only selected fields
        REPD = pd.read_csv(r'Renewable_Energy_Planning_Data.csv', encoding='latin1', usecols=fields)

        # Clear column headings of spaces and anything after brackets/parenthesis to make calling them easier e.g. in filtering
        REPD.columns = REPD.columns.str.replace(' ', '')
        REPD.columns = REPD.columns.str.replace(r"\(.*\)", "")

        # Filter to offshore wind in the technology type column
        filter_list_technology = ['Wind Onshore']
        REPD_Wind = REPD[REPD.TechnologyType.isin(filter_list_technology)]

        # Filter to operational in the development status column
        filter_list_status = ['Operational']
        REPD_Wind = REPD_Wind[REPD_Wind.DevelopmentStatus.isin(filter_list_status)]

        # Remove leading and trailing spaces to make it easier to match across data tables
        REPD_Wind['SiteName'] = REPD_Wind['SiteName'].str.strip()
        return REPD_Wind
    except Exception as e:
        print("Exception in get_REPD_Wind_df: {}".format(str(e)))
        print(traceback.format_exc())

In [5]:
def main():
    try:
        # just extracting the data
        Wikipedia = pd.read_excel(r'Total_onshore.xlsx')
        REPD_Wind = get_REPD_Wind_df()
        # converting cooords to  lat/lon we canuse  to calculate distance
        REPD_Wind['gov_coords'] = REPD_Wind.apply(lambda row: convert_uk_grid_to_latlon(row), axis=1)
        Wikipedia['wiki_coords'] = Wikipedia['Coordinates'].apply(formatted_string_to_latlon)

        Wikipedia.to_csv('Wiki.csv', index=False)
        REPD_Wind.to_csv('REPD_Wind.csv')

        # Priority 1:exact matchjooin
        merged_left = pd.merge(left=REPD_Wind, right=Wikipedia, how='left', left_on='SiteName', right_on='Name')

        # splitting the df into 2 dataframes: with 'pair' after  join (matched)and not matched
        # we are going tofind pairsfor 'not matched'
        matched = merged_left[merged_left['Name'].str.len() > 0]
        not_matched = merged_left[merged_left['Name'].isnull()]
        # matched['distance'] = matched.apply(lambda x: get_distance(x['gov_coords'], x['wiki_coords']), axis=1)

        series_to_join = []
        # iterating non-matched df
        for repd_index, repd_row in not_matched.iterrows():
            min_distance = 100000
            matches_distance = 100000
            row_detected = None
            match = None
            for wiki_index, wiki_row in Wikipedia.iterrows():

                # calculating distance with each wiki object
                distance = get_distance(repd_row['gov_coords'], wiki_row['wiki_coords'])

                # checking for 'inner matches'
                if repd_row['SiteName'] in wiki_row['Name'] or wiki_row['Name'] in repd_row['SiteName']:
                    if distance < matches_distance:
                        match = wiki_row

                # checling for ''common words
                elif find_commonwords_num(repd_row['SiteName'], wiki_row['Name'], words_to_ignore) > 1:
                    if distance < matches_distance:
                        match = wiki_row
                # update values if it's close
                if distance < min_distance:
                    min_distance = distance
                    row_detected = wiki_row
            print('*')
            print(repd_row['SiteName'])
            # let's check if closest objects haveat least onecommon  word
            common_words = find_commonwords_num(repd_row['SiteName'], row_detected['Name'], words_to_ignore)
            if common_words == 0 and match is not None:
                # it not - lt's use aclosest match
                print(match['Name'])
                row_to_join = match

            elif common_words > 0:
                print(row_detected['Name'])
                row_to_join = row_detected
            else:
                print('Not found....')
                series_to_join.append(repd_row)
                continue
            # updating values in initial series
            for k in row_to_join.keys():
                repd_row[k] = row_to_join[k]
            series_to_join.append(repd_row)

        # jet's join a list to df and join with amatched df
        new_matched = pd.DataFrame(series_to_join)
        new_matched = matched.append(new_matched)
        # filter out some columns
        new_matched = new_matched[
            [x for x in new_matched.columns if x not in ['Coordinates', 'wiki_coords', 'gov_coords']]]
        new_matched.to_csv("final_joined.csv", index=False)


    except Exception as e:
        print("Exception in main: {}".format(str(e)))
        print(traceback.format_exc())


if __name__ == '__main__':
    main()

Exception in get_REPD_Wind_df: 'DataFrame' object has no attribute 'DevelopmentStatus'
Traceback (most recent call last):
  File "/var/folders/vc/gszmszx94qn2yq038bgz88mr0000gn/T/ipykernel_75828/3279753695.py", line 39, in get_REPD_Wind_df
    REPD_Wind = REPD_Wind[REPD_Wind.DevelopmentStatus.isin(filter_list_status)]
  File "/Users/oliverlysaght/anaconda3/lib/python3.10/site-packages/pandas/core/generic.py", line 6204, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'DevelopmentStatus'

Exception in main: 'NoneType' object has no attribute 'apply'
Traceback (most recent call last):
  File "/var/folders/vc/gszmszx94qn2yq038bgz88mr0000gn/T/ipykernel_75828/1932107905.py", line 7, in main
    REPD_Wind['gov_coords'] = REPD_Wind.apply(lambda row: convert_uk_grid_to_latlon(row), axis=1)
AttributeError: 'NoneType' object has no attribute 'apply'

